In [ ]:
import pandas as pd
import warnings
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from plotly.subplots import make_subplots
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

# Предварительное исследование данных и предобработка
Для начала посмотрим на данные и их качество

In [ ]:
payments = pd.read_csv('/content/drive/MyDrive/Тестовое Маркетинговый аналитик/payments.csv',
                          parse_dates=['payment_date'])
persents = pd.read_csv('/content/drive/MyDrive/Тестовое Маркетинговый аналитик/persents.csv')
regs     = pd.read_csv('/content/drive/MyDrive/Тестовое Маркетинговый аналитик/registrations.csv',
                          parse_dates=['created_date'])
budget = pd.read_excel('/content/drive/MyDrive/Тестовое Маркетинговый аналитик/Рекламный бюджет январь 2021.xlsx')

### Платежи

In [ ]:
payments.head()

,payment_date,Payment_types,real_cost,account_id
0,2021-01-01 00:00:56,Payment_system_5,9.65,2907221
1,2021-01-01 00:01:48,Payment_system_5,0.96,3228373
2,2021-01-01 00:03:42,Payment_system_5,3.03,318552
3,2021-01-01 00:04:27,Payment_system_2,99.99,3832817
4,2021-01-01 00:06:41,Payment_system_5,1.43,7229767


In [ ]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303614 entries, 0 to 303613
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   payment_date   303614 non-null  datetime64[ns]
 1   Payment_types  303614 non-null  object        
 2   real_cost      303614 non-null  float64       
 3   account_id     303614 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 9.3+ MB


In [ ]:
payments[['real_cost', 'account_id']].describe()

,real_cost,account_id
count,303614.00,303614.00
mean,11.69,4063360.57
std,17.03,2587446.28
min,0.00,24.00
25%,1.10,2162915.00
50%,4.76,3755245.00
75%,11.27,4873232.75
max,261.64,9999944.00


In [ ]:
payments.loc[payments.duplicated()]

,payment_date,Payment_types,real_cost,account_id
35708,2021-02-15 14:40:13,Payment_system_3,1.99,1827251
43293,2021-02-26 17:35:31,Payment_system_5,45.20,3945214
262705,2021-11-15 16:12:44,Payment_system_4,0.47,4720239
268378,2021-11-21 03:22:42,Payment_system_2,62.00,8768141


В данных присутствуют 4 дублирующихся платежа, что, скорее всего, является багом (проведены одновременно с точностью до секунды), поэтому избавимся от этих строк.

В остальном данные вылядят корректными

In [ ]:
payments = payments.drop_duplicates()

### Проценты

In [ ]:
persents

,Unnamed: 0,Payment_types,"Share, %"
0,0,Payment_system_1,20.70
1,1,Payment_system_2,15.00
2,2,Payment_system_3,58.50
3,3,Payment_system_4,0.00
4,4,Payment_system_5,24.00


В таблице присутствует лишний столбец, который, вероятно, является последствием сохранения датафрейма в формате csv без игнорирования индексов - избавимся от него, так как он не несет какой-либо информации

In [ ]:
persents = persents.drop(columns=['Unnamed: 0'])

### Регистрации

In [ ]:
regs.head()

,Unnamed: 0,account_id,created_date,campaign
0,0,3842380.00,2021-01-01,BRA_MS1_install
1,1,3842381.00,2021-01-01,NaN
2,2,3842382.00,2021-01-01,NaN
3,3,3842383.00,2021-01-01,NaN
4,4,3842384.00,2021-01-01,BRA_MS1_install


In [ ]:
regs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89144 entries, 0 to 89143
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    89144 non-null  int64         
 1   account_id    89141 non-null  float64       
 2   created_date  89144 non-null  datetime64[ns]
 3   campaign      36283 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 2.7+ MB


In [ ]:
regs.loc[regs.duplicated()]

,Unnamed: 0,account_id,created_date,campaign


In [ ]:
regs.loc[regs.duplicated(subset=['account_id'])]

,Unnamed: 0,account_id,created_date,campaign
76130,76130,NaN,2021-01-26,NaN
78464,78464,NaN,2021-01-27,NaN


In [ ]:
regs[['Unnamed: 0', 'account_id']].describe()

,Unnamed: 0,account_id
count,89144.00,89141.00
mean,44571.50,3886951.08
std,25733.80,25734.17
min,0.00,3842380.00
25%,22285.75,3864665.00
50%,44571.50,3886950.00
75%,66857.25,3909237.00
max,89143.00,3931524.00


В данных есть несколько проблем, которые необходимо решить:
1. Столбец с индексами - удалим
2. Пропущенные значения в столбце campaign, что связано, предположительно, с тем, что данные пользователи являются органикой (пришли сами) - избавимся от таких строк
3. Пропущенные значения в id аккаунтов, что, видимо, является багом - удаляем
4. Тип float у account_id - просто неприятно глазу, меняем на int)

In [ ]:
regs = regs.drop(columns=['Unnamed: 0'])
regs = regs.dropna(subset=['campaign', 'account_id'])
regs['account_id'] = regs['account_id'].astype(int)

### Рекламный бюджет Январь 2021

In [ ]:
budget

,media_source,Campaign_type,Target,"Plan, USD",Installs,"Spend, USD","Deviation, USD"
0,Media_source_1,install,Brazil,100,10460,99.12,0.88
1,NaN,purchase,Brazil,400,1081,398.67,1.33
2,NaN,install,Russia,200,11894,246.30,-46.30
3,NaN,purchase,Russia,500,1441,616.77,-116.77
4,NaN,install,Ukrane,100,6424,99.43,0.57
5,NaN,purchase,Ukrane,500,242,348.13,151.87
6,Media_source_2,install,Brazil,100,152,81.15,18.85
7,NaN,install,English_speaking,100,371,99.95,0.05
8,Total,NaN,NaN,2000,32065,1989.52,10.48


Необходимо заполнить пропуски в столбце media_source, а также удалить последнюю строку (так как сводные данные, в нашем случае, не нужны)

In [ ]:
budget = budget.iloc[:-1]
budget.iloc[1:6, 0] = 'Media_source_1'
budget.iloc[7, 0] = 'Media_source_2'

Также, необходимо добавить столбец с названием кампании, соответствующего названиям кампаний из таблицы с данными о регистрации, для этого посомтрим на них

In [ ]:
regs['campaign'].unique()

array(['BRA_MS1_install', 'RUS_MS1_install', 'UKR_MS1_install',
       'BRA_MS1_purchase', 'RUS_MS1_purchase', 'PR_youtube_bloger',
       'UKR_MS1_purchase', 'ENG_MS2_install', 'BRA_MS2_install', 'our_fb'],
      dtype=object)

In [ ]:
countries = {
    'Brazil': 'BRA',
    'Russia': 'RUS',
    'Ukrane': 'UKR',
    'English_speaking': 'ENG'
}
sources = {
    'Media_source_1': 'MS1',
    'Media_source_2': 'MS2',
}
budget['campaign'] = budget.apply(lambda x: f"{countries[x['Target']]}_{sources[x['media_source']]}_{x['Campaign_type']}", axis=1)
budget

,media_source,Campaign_type,Target,"Plan, USD",Installs,"Spend, USD","Deviation, USD",campaign
0,Media_source_1,install,Brazil,100,10460,99.12,0.88,BRA_MS1_install
1,Media_source_1,purchase,Brazil,400,1081,398.67,1.33,BRA_MS1_purchase
2,Media_source_1,install,Russia,200,11894,246.30,-46.30,RUS_MS1_install
3,Media_source_1,purchase,Russia,500,1441,616.77,-116.77,RUS_MS1_purchase
4,Media_source_1,install,Ukrane,100,6424,99.43,0.57,UKR_MS1_install
5,Media_source_1,purchase,Ukrane,500,242,348.13,151.87,UKR_MS1_purchase
6,Media_source_2,install,Brazil,100,152,81.15,18.85,BRA_MS2_install
7,Media_source_2,install,English_speaking,100,371,99.95,0.05,ENG_MS2_install


Была обнаружена рекламная кампания с названием our_fb, так как ее нет в таблице с бюджетом, предполагаю, что на данную кампанию не предусмотерны расходы, скорее всего это реклама в социальных сетях компании, поэтому в дальнейшем анализе ее рассматривать не будем.

Еще одна кампания - PR_youtube_bloger, так как мы не располагаем данными о бюджете данной кампании, ее рассматривать также не будем

In [ ]:
regs = regs[-regs['campaign'].isin(['our_fb', 'PR_youtube_bloger'])]
regs.shape

(34440, 3)

Тут замечу, что число установок в данных registrations.csv и отчете о рекламном бюджете за январь, который предоставил коллега, различаются. Возможно, причино в том, что данные получены из разных источников, но так как все дальнейщшие расчеты будут опираться на выгрузки из БД, будем считать именно эти цифры истинными.

У нас имеются данные о покупках пользователей, а также комиссии платежных систем. Переменная real_cost в платежах является "сконвертированной суммой", но мы не имеем информации о том, учитывается ли в данной сумме комисия платежной системы или это просто сконвертированные в доллары суммы в местной авлюте. Так как нам предоставлены данные о комиссии, предположим, что из этих сумм также необходимо вычесть комисиию платежной системы, таким образом получим чистый доход

In [ ]:
payments = payments.merge(persents, on = 'Payment_types', how='left')
payments['revenue'] = payments['real_cost'] * (1 - (payments['Share, %']/100))
payments.head()

,payment_date,Payment_types,real_cost,account_id,"Share, %",revenue
0,2021-01-01 00:00:56,Payment_system_5,9.65,2907221,24.00,7.33
1,2021-01-01 00:01:48,Payment_system_5,0.96,3228373,24.00,0.73
2,2021-01-01 00:03:42,Payment_system_5,3.03,318552,24.00,2.30
3,2021-01-01 00:04:27,Payment_system_2,99.99,3832817,15.00,84.99
4,2021-01-01 00:06:41,Payment_system_5,1.43,7229767,24.00,1.09


Проверим, есть ли в данных о регистрациях аккаунты зарегистрированные не в январе 2021

In [ ]:
regs[(regs['created_date'].dt.year != 2021) | (regs['created_date'].dt.month != 1)]

,account_id,created_date,campaign


Таких аккаунтов нет, проверим также есть ли покупки раньше января

In [ ]:
payments[payments['payment_date'].dt.year < 2021]

,payment_date,Payment_types,real_cost,account_id,"Share, %",revenue


Также необходимо соединить данные о платежах с информацией о регистрации, чтобы, во-первых, исключить платежи с аккаунтов, зарегистрированных не в январе 2021, а во вторых получить данные о рекламных кампаниях, с которых пришли пользователи.

Кроме этого, преобразуем столбец с датой платежа в более короткий формат, так как в дальнейшем нам не нужна более детальная информация, чем день регистрации, а также добавим столбец с днем рекламной кампании (предполагаем, что все кампании начались с 1 января, так как опровергающей информации не дано) и днем с регистрации аккаунта

In [ ]:
final_df = payments.merge(regs, on='account_id', how = 'inner') # оставляем только аккаунты с платежами
final_df['payment_date'] = final_df['payment_date'].dt.normalize()
final_df['days_of_campaign'] = (final_df['payment_date'] - pd.to_datetime('2021-01-01')).dt.days + 1
final_df['acc_days'] = (final_df['payment_date'] - final_df['created_date']).dt.days + 1
final_df.head()

,payment_date,Payment_types,real_cost,account_id,"Share, %",revenue,created_date,campaign,days_of_campaign,acc_days
0,2021-01-01,Payment_system_5,2.45,3861811,24.00,1.86,2021-01-07,RUS_MS1_purchase,1,-5
1,2021-01-01,Payment_system_5,2.45,3861811,24.00,1.86,2021-01-07,RUS_MS1_purchase,1,-5
2,2021-01-01,Payment_system_5,9.62,3844647,24.00,7.31,2021-01-01,BRA_MS1_install,1,1
3,2021-01-01,Payment_system_5,9.62,3844647,24.00,7.31,2021-01-01,BRA_MS1_install,1,1
4,2021-01-02,Payment_system_5,0.96,3845613,24.00,0.73,2021-01-02,RUS_MS1_purchase,2,1


Видим, что у нас есть платежи, которые произошли раньше, чем аккаунт был зарегистрироыван, что определенно явлется багом и от таких строк необходимо избавиться

In [ ]:
final_df = final_df[final_df['acc_days'] >= 1]
final_df.shape

(670, 10)

Итоговый датафрейм содержит 670 платежей, можем переходить к вопросам Богдана

# Вопросы Богдана

## Определить оптимальный срок оценки окупаемости рекламных кампаний.

У нас есть 8 рекламных кампаний, проверим срок окупаемости каждой из них. Для этого:
- Рассчитаем ежедневный ROAS (коэффициент возврата рекламных затрат) каждой из кампаний
- Определим через сколько дней РК окупилась и построим график

Далее рассчитаем средний срок окупаемости РК - он и будет оптимальным сроком оценки окупаемости новых рекламных кампаний.


In [ ]:
fig = go.Figure()
max_days = 0
compaigns_PP = []
all_camp_roas = pd.DataFrame()
for campaign in budget['campaign'].unique():
    campaign_data = final_df[final_df['campaign'] == campaign].sort_values('payment_date') # сортируем, чтобы правильно рассчитать кумулятивный доход
    campaign_data['cum_rev'] = campaign_data['revenue'].cumsum()
    campaign_roas = campaign_data.groupby('days_of_campaign', as_index=False).agg({'cum_rev': 'last'}) # получаем последнее кумулятивное значение за день
    campaign_budget = budget[budget['campaign'] == campaign]['Spend, USD'].item() # бюджет кампании
    campaign_roas['budget'] = campaign_budget
    campaign_roas['ROAS'] = campaign_roas['cum_rev'] / campaign_roas['budget']
    campaign_roas['campaign'] = campaign

    all_camp_roas = pd.concat([all_camp_roas, campaign_roas], ignore_index=True)

    fig.add_trace(go.Scatter(
    x=campaign_roas['days_of_campaign'],
    y=campaign_roas['ROAS'],
    mode='lines',
    name=campaign
    ))

    if campaign_roas['ROAS'].max() < 1: # если ROAS за все время так и не достиг 1 - кампания не окупилась
        print(f'Кампания {campaign} не окупилась')
        continue
    max_days = max(max_days, campaign_roas['days_of_campaign'].max()) # для графика
    campaign_PP = campaign_roas[campaign_roas['ROAS'] >= 1].head(1)['days_of_campaign'].item() # день кампании, когда ROAS в первый раз больше 1 (срок окупаемости)
    compaigns_PP.append(campaign_PP)
    print(f'Кампания {campaign} окупилась на {campaign_PP} день')

print('')

# Добавляем горизонтальную линию на уровне 1 - точка окупаемости
fig.add_shape(
    type="line",
    x0=0,
    x1=max_days,
    y0=1,
    y1=1,
    line=dict(color="red", dash="dash")
)

fig.update_layout(
    title='Окупаемость рекламных кампаний',
    xaxis_title='День от начала кампании',
    yaxis_title='ROAS',
    showlegend=True,
    template='plotly_dark'
)
fig.show()


Кампания BRA_MS1_install окупилась на 59 день
Кампания BRA_MS1_purchase окупилась на 211 день
Кампания RUS_MS1_install окупилась на 135 день
Кампания RUS_MS1_purchase окупилась на 302 день
Кампания UKR_MS1_install окупилась на 289 день
Кампания UKR_MS1_purchase окупилась на 183 день
Кампания BRA_MS2_install не окупилась
Кампания ENG_MS2_install не окупилась



В нашем случае есть кампании, которые совсем не окупились, по графику видно, что они в принципе приносят маленький доход, поэтому мы исключаем их из анализа.

Оптимальный срок оценки окупаемости можно определить в зависимости от целей: если важно, чтобы инвестиции в рекламу окупались как можно скорее, стоит взять среднее или медиану. Если же критично оценить окупаемость в принципе, вне зависимости от её длительности, можно опираться на максимальный срок.

Однако, по нашему графику видно, что кампании делятся на три сегмента:
- Успешные — окупаются и в дальнейшем приносят большой доход.

- Окупившиеся — окупаются, но приносят небольшой доход.

- Не окупившиеся — совсем не окупаются.

Для оценки успеха нас интересует первый сегмент, так как именно он приносит максимальный доход. В большинстве случаев, чем быстрее окупаемость, тем выше ROAS и, в последствии, ROI (доход на вложенный рубль). Таким образом, ориентироваться на максимальный срок не имеет смысла, поскольку долгие сроки окупаемости характерны для менее успешных кампаний.

Лучше использовать медиану, так как она менее чувствительна к выбросам. Например, кампания с окупаемостью в 59 дней сильно занизит среднее значение, а это может привести к исключению потенциально прибыльных кампаний, которые окупаются чуть дольше.

In [ ]:
PP_median = np.median(compaigns_PP)
PP_median

197.0

В данном случае медиана составляет **197 дней**, и это оптимальный срок оценки окупаемости в нашем случае.

## Определить наиболее успешные рекламные кампании, типы рекламных кампаний и рекламную сеть на момент оценки окупаемости

По данным из предыдущего раздела сразу можно сказать, что рекламная сеть 2 наименее успешна (Media_source_2).

Посмотрим также на сами кампании и их типы

In [ ]:
# ближайшее к дню оценки окупаемости ROAS
roas_by_camp = all_camp_roas[all_camp_roas['days_of_campaign'] <= PP_median] \
  .groupby('campaign', as_index=False) \
  .agg({'ROAS': 'last'}) \
  .sort_values('ROAS', ascending=False)

roas_by_camp

,campaign,ROAS
0,BRA_MS1_install,1.88
4,RUS_MS1_install,1.14
7,UKR_MS1_purchase,1.09
1,BRA_MS1_purchase,0.99
5,RUS_MS1_purchase,0.84
6,UKR_MS1_install,0.68
3,ENG_MS2_install,0.26
2,BRA_MS2_install,0.15


Наиболее успешная кампания - **BRA_MS1_install**

При этом, окупились еще две - RUS_MS1_install и UKR_MS1_purchase

In [ ]:
roas_by_camp['type'] = roas_by_camp['campaign'].str.split('_').str[-1]
roas_by_camp.groupby('type', as_index=False).agg({'ROAS': 'median'})

,type,ROAS
0,install,0.68
1,purchase,0.99


Медианное значение ROAS кампаний, направленных на покупки выше, чем на установки, так как все такие кампании имеют неплохие результаты. При этом, кампании направленные на установку, есть как очень успешные, так и совсем неуспешные, поэтому итоговое значение низкое.

В целом, предполагаю, что так как цели у кампаний разные, то и метрики для оценки их успешности должны быть разными. Для покупки - это конверсия в покупку, для установки - конверсия в установку, но у нас недостаточно данных для расчета конверсии в установку.

В целом, есть метрики помимо ROAS, которые мы можем оценить на момент оценки окупаемости, но это мы рассмотрим далее

## Рассчитать накопительный ARPU (average revenue per user) успешных рекламных кампаний, оптимизированных под покупки, на 7, 14, 30 день жизни аккаунтов, для прогноза окупаемости новых рекламных кампаний.

Для расчета ARPU нам нужны все пользователи, в том числе те, которые не совершали покупки, посчитаем их количество для каждой рекламной кампании, оптимизированной под продажи

In [ ]:
regs['type'] = regs['campaign'].str.split('_').str[-1]
arpu_purchase = regs[regs['type'] == 'purchase']
campaign_counts = arpu_purchase.groupby('campaign', as_index=False).agg({'account_id': 'count'})
campaign_counts

,campaign,account_id
0,BRA_MS1_purchase,1139
1,RUS_MS1_purchase,1581
2,UKR_MS1_purchase,260


In [ ]:
final_df['type'] = final_df['campaign'].str.split('_').str[-1]
target_days = [7, 14, 30]
results = []
target_results = []

payments_purchase = final_df[final_df['type'] == 'purchase']

for campaign in payments_purchase['campaign'].unique():
    campaign_data = payments_purchase[payments_purchase['campaign'] == campaign]
    campaign_data = campaign_data.groupby(['campaign', 'acc_days'], as_index=False).agg({'revenue': 'sum'})
    campaign_data['revenue_cum'] = campaign_data['revenue'].cumsum()
    campaign_data['users'] = campaign_counts[campaign_counts['campaign'] == campaign]['account_id'].item()
    campaign_data['arpu'] = campaign_data['revenue_cum'] / campaign_data['users']

    # Поиск ближайших дней
    indices = [
        campaign_data.loc[campaign_data['acc_days'] <= day].index.max()
        for day in target_days
    ]

    target_data = campaign_data.loc[indices].copy()
    target_data['target_day'] = target_days
    target_data['campaign'] = campaign

    target_results.append(target_data)
    results.append(campaign_data)

final_results = pd.concat(results, ignore_index=True)
target_results = pd.concat(target_results, ignore_index=True)


In [ ]:
# Функция для добавления графиков
def add_campaign_bars(data, col, row, show_legend):
    for campaign in target_results['campaign'].unique():
        fig.add_trace(go.Bar(
            x = data[data['campaign'] == campaign]['campaign'],
            y = data[data['campaign'] == campaign]['arpu'],
            text = data[data['campaign'] == campaign]['arpu'].round(2),
            textposition='outside',
            name = campaign,
            legendgroup = campaign,
            marker_color = campaign_colors[campaign],
            showlegend = show_legend
        ), row=row, col = col)

campaign_colors = {campaign: color for campaign, color in zip(target_results['campaign'].unique(), ['#1f77b4', '#ff7f0e', '#2ca02c'])}

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['7 дней', '14 дней', '30 дней'],
    shared_yaxes=True
)
data_7 = target_results[target_results['target_day'] == 7]
data_14 = target_results[target_results['target_day'] == 14]
data_30 = target_results[target_results['target_day'] == 30]

add_campaign_bars(data_7, 1, 1, True)
add_campaign_bars(data_14, 2, 1, False)
add_campaign_bars(data_30, 3, 1, False)

fig.update_layout(
    title='Сравнение ARPU по кампаниям на 7, 14 и 30 день',
    yaxis_title='ARPU',
    showlegend=True,
    xaxis=dict(tickvals=[], showticklabels=False),
    xaxis2=dict(tickvals=[], showticklabels=False),
    xaxis3=dict(tickvals=[], showticklabels=False),
    xaxis_title='',
    template='plotly_dark'
)

fig.show()


In [ ]:
# Визуализация данных по всем дням
fig = px.line(
    final_results,
    x='acc_days',
    y='arpu',
    color='campaign',
    title='Накопительный ARPU по всем дням',
    labels={'acc_days': 'Дни с момента запуска', 'arpu': 'ARPU'},
    color_discrete_map=campaign_colors
)

fig.update_layout(
    xaxis=dict(tickmode='linear', tick0=0, dtick=10),
    yaxis=dict(title='ARPU'),
    legend_title_text='Кампании',
    template='plotly_dark'
)

fig.show()

## Рассчитать фактический CPI (cost per install) для каждой рекламной кампании

CPI возможно рассчитать на основе данных из отчета коллеги, так как там есть данные, как о расходах на кампании, так и о количестве установок, однако, данные о кличестве установок различаются и, как уже оговаривалось ранее, опираться будем именно на данные из БД, поэтому пересчитаем количество установок по кампаниям и соответствующие CPI.

In [ ]:
install_counts = regs.groupby('campaign', as_index=False).agg({'account_id': 'count'})
install_counts = install_counts.merge(budget[['campaign', 'Spend, USD']], on='campaign', how = 'left')
install_counts['CPI'] = install_counts['Spend, USD'] / install_counts['account_id']
install_counts = install_counts.sort_values('CPI', ascending=False)
install_counts

,campaign,account_id,"Spend, USD",CPI
7,UKR_MS1_purchase,260,348.13,1.34
2,BRA_MS2_install,152,81.15,0.53
5,RUS_MS1_purchase,1581,616.77,0.39
1,BRA_MS1_purchase,1139,398.67,0.35
3,ENG_MS2_install,407,99.95,0.25
4,RUS_MS1_install,12649,246.30,0.02
6,UKR_MS1_install,6790,99.43,0.01
0,BRA_MS1_install,11462,99.12,0.01


In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = install_counts['CPI'],
    y = install_counts['campaign'],
    orientation = 'h',
    marker = dict(color='#ff7f0e', line=dict(color='#ff7f0e', width=1)),
    text = install_counts['CPI'].round(2),
    textposition='outside'
))

# Настраиваем оформление графика
fig.update_layout(
    title='Стоимость за установку (CPI) по кампаниям',
    xaxis_title='CPI',
    yaxis_title='Кампании',
    xaxis=dict(tickformat='.2f'),
    height=500,
    margin=dict(l=150),
    template='plotly_dark'
)

fig.show()

Также можно объединить графики ARPU и CPI для рекламных кампаний, ориентированных на покупки

In [ ]:
final_results = final_results.merge(install_counts[['campaign', 'CPI']], on='campaign', how='left')
campaigns = final_results['campaign'].unique()

campaign_colors = {
    campaigns[0]: '#1f77b4',
    campaigns[1]: '#ff7f0e',
    campaigns[2]: '#2ca02c',
}


figs = []
for campaign in campaigns:
    campaign_data = final_results[final_results['campaign'] == campaign]
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=campaign_data['acc_days'],
        y=campaign_data['arpu'],
        mode='lines',
        name=f'ARPU - {campaign}',
        line=dict(color=campaign_colors.get(campaign, '#1f77b4'))
    ))

    # Добавляем линию CPI (красная)
    fig.add_trace(go.Scatter(
        x=campaign_data['acc_days'],
        y=campaign_data['CPI'],
        mode='lines',
        name=f'CPI - {campaign}',
        line=dict(color='red')
    ))

    fig.update_layout(
        title=f'ARPU и CPI для кампании {campaign}',
        xaxis_title='Дни с момента запуска',
        yaxis_title='Значения',
        legend_title='Метрические показатели',
        template='plotly_dark'
    )
    figs.append(fig)

for fig in figs:
    fig.show()

# Дополнительные метрики для оценки успешности кампаний

Здесь я бы рассматривала разные типы кампаний отдельно, так как они имеют различные цели.

Для кампаний, ориентированных на установку, в большей степени важна конверсия в установку, при этом скорость окупаемости может быть ниже. Здесь также можно анализировать коэффициент удержания, DAU и MAU, но как и для конверсии, у нас недостаточно данных.

Для кампаний, направленных на покупки, на ранних стадиях в больлшей стпени важна конверсия именно в покупку, ARPU и ARPPU, CPO (стоимость за покупку в нашем случае)

Посомтрим на ежемесячные (кумулятивные) конверсии в покупку, чтобы оценить возможность определения успешности кампаний до оценки окупаемости

In [ ]:
cr_df = (final_df[final_df['type'] == 'purchase']
         .groupby(['account_id'], as_index=False)
         .agg({'payment_date': 'first'})
         .assign(payment_month=lambda df: df['payment_date'].dt.month))

cr_df = cr_df.merge(final_df[['account_id', 'campaign']], on='account_id', how='left')

cr_df = (cr_df.groupby(['payment_month', 'campaign'], as_index=False)
         .agg(first_pays=('account_id', 'count'))
         .assign(cumulative_first_pays=lambda df: df.groupby('campaign')['first_pays'].cumsum()))

cr_df = (cr_df.merge(install_counts[['campaign', 'account_id']], on='campaign', how='left')
              .rename(columns={'account_id': 'user_count'})
              .assign(cr=lambda df: df['cumulative_first_pays'] / df['user_count']))

In [ ]:
colors = ['#ff7f0e','#1f77b4', '#2ca02c']
fig = px.line(cr_df,
              x='payment_month',
              y='cr',
              color='campaign',
              labels={'payment_month': 'Месяц', 'cr': 'CR в продажу'},
              title='CR в продажу для кампаний ориентированных на продажу',
              color_discrete_sequence=colors,
              template='plotly_dark')
fig.show()


Сильная разница в конверсиях видна уже через 2 месяца, после чего тренд меняется несильно. Поэтому, данную метрику можно использовать на ранних этапах, вкупи с другими показателями (ARPU, CPI)